# Project 1 - Credit risk for mortages

##  2.1.1

We wish to make a model that decides whether or not to grant a loan to an individual based on relevant data, our goal is to maximize profit. Our utility function is defined as:

| Utility | Grant loan ($a_1$) | Don't grant loan ($a_2$) |
|----------------|-------------|------------------|
| Loan repaid ($\omega_1$)    | $m((1+r)^n-1)$       | 0                |
| Loan defaulted ($\omega_2$) | $-m$                 | 0                |

Thus our expected utility function is defined as our predicted probability of the person repaying the loan times the return on investment, minus our predicted probability that the person will default on the loan times the investment.

$$E(U|\textrm{Grant loan}) = P(\omega_1)\cdot m((1+r)^n - 1) + P(\omega_2)\cdot (-m),$$
$$E(U|\textrm{Don't grant loan}) = 0$$

In [1]:
def expected_utility(self, X):
    pr_1, pr_2 = self.predict_proba(X).T
    U_1 = X['amount']*( (1+self.interest_rate)**X['duration'] - 1)
    U_2 = -X['amount']
    return pr_1*U_1 + pr_2*U_2

We grant a loan when:
$$E(U|\textrm{Grant loan}) > E(U|\textrm{Don't grant loan})$$

We considered different models for predicting the probability, and made our final decision later in the process.

## 2.1.2

We implemented `́NameBanker.fit()` and `NameBanker.predict_proba()` by calling on our classifiers own `fit()` and `predict_proba()` functions.

In [2]:
def fit(self, X, y):
    self.classifier = RandomForestClassifier(n_estimators=self.n_estimators)
    self.classifier.fit(X, y)

def predict_proba(self, X):
    return self.classifier.predict_proba(X.to_frame().transpose())

For our `KNearestNeighborsBanker` we standardized all the non-categorical features before fitting the data, and before making new predictions.

## 2.1.3

As noted earlier we grant a loan if
$$E(U|\textrm{Grant loan}) > E(U|\textrm{Don't grant loan})$$
otherwise we don't grant the loan.

In [3]:
def get_best_action(self, X):
    for exp_utility in self.expected_utility(X):
        if exp_utility > 0:
            return 1
        else:
            return 2

## 2.1.4

For model selection we tried different classifiers for comparison; KNearestNeighbors, LogisticRegression and RandomForest, with varying hyperparameters, and we also compared their performance to the RandomBanker (grants loans randomly) and the YesBanker (always grants loans). We kept the original interest rate of 0.5% per month as with the new rate of 5% the YesBanker was one of the best performing models and even the RandomBanker was profitting. We felt it made little sense to optimize for an unrealistically high interest rate.

In [2]:
# Slightly modified TestLending.py so it runs faster and we can call the
# model checking as a function model_check
from TestLending import X, model_check
from banker import run
from randomforestbanker import RandomForestBanker
from logisticbanker import LogisticBanker
from kneighborsbanker import KNeighborsBanker
from randombanker import RandomBanker
from yesbanker import YesBanker

In [5]:
interest_rate = 0.005

print("RandomBanker: ")
model_check(X,
            RandomBanker(interest_rate=interest_rate),
            interest_rate=interest_rate)

print("YesBanker: ")
model_check(X,
            YesBanker(interest_rate=interest_rate),
            interest_rate=interest_rate)

print("LogisticBanker: ")
model_check(X,
            LogisticBanker(interest_rate=interest_rate),
            interest_rate=interest_rate)

for k in [1, 5, 15, 25, 35]:
    print(f"KNeighborsBanker with k={k}:")
    model_check(X,
                KNeighborsBanker(interest_rate=interest_rate, k=k),
                interest_rate=interest_rate)

for n in range(25, 151, 25):
    print(f"RandomForestBanker with estimators={n}:")
    model_check(X,
                RandomForestBanker(interest_rate=interest_rate, n_estimators=n),
                interest_rate=interest_rate)

RandomBanker: 
-88067.00505039457
YesBanker: 
-180882.2863569728
LogisticBanker: 
4819.749239562388
KNeighborsBanker with k=1:
-109034.30341244252
KNeighborsBanker with k=5:
-29172.39059167475
KNeighborsBanker with k=15:
-7052.263029378393
KNeighborsBanker with k=25:
-3290.8560267370713
KNeighborsBanker with k=35:
-3039.8953654492716
RandomForestBanker with estimators=25:
5067.98836574307
RandomForestBanker with estimators=50:
4959.466365119092
RandomForestBanker with estimators=75:
5971.469270722375
RandomForestBanker with estimators=100:
6037.942862414185
RandomForestBanker with estimators=125:
6313.146526087533
RandomForestBanker with estimators=150:
5949.893176655319


In [6]:
# n_estimators = 100 and k = 15 by default
run()

RandomForestBanker (714.1536225547587, 1062.3282027530518)
LogisticBanker (433.97574560359425, 2783.9969836057344)
KNeighborsBanker (96.85603287170403, 1938.901379761263)
RandomBanker (-10664.503410547937, 13057.236182011911)
YesBanker (-17924.250714848473, 11782.002412033245)


From these results we would pick `RandomForestBanker` with an `n_estimators` somewhere between 75 and 150. With Hesse's `banker.py` program we can see that `RandomForestBanker` has both the highest utility and the lowest standard deviation of all the bankers. However, we have not used a separate test set to evaluate the final performance of our model, which we should do as we are using the test results to do a final model selection.

We made all our bankers compatible with both Dimitrakakis' `TestLending.py` and Hesse's `banker.py`. However, in this notebook we used a modified version of `TestLending.py` that runs faster and put the main program in a function `model_check()` with no functional differences from the original.

For some reason `KNeighborsBanker` performs way worse on `TestLending.py` than on `banker.py` and we could not figure out why, but all the other models performed similarily between the programs.

In [4]:
X

,duration,amount,installment,residence time,age,credits,persons,repaid,checking account balance_A12,checking account balance_A13,...,property_A124,other installments_A142,other installments_A143,housing_A152,housing_A153,job_A172,job_A173,job_A174,phone_A192,foreign_A202
0,6,1169,4,4,67,2,1,1,0,0,...,0,0,1,1,0,0,1,0,1,0
1,48,5951,2,2,22,1,1,2,1,0,...,0,0,1,1,0,0,1,0,0,0
2,12,2096,2,3,49,1,2,1,0,0,...,0,0,1,1,0,1,0,0,0,0
3,42,7882,2,4,45,1,2,1,0,0,...,0,0,1,0,1,0,1,0,0,0
4,24,4870,3,4,53,2,2,2,0,0,...,1,0,1,0,1,0,1,0,0,0
5,36,9055,2,4,35,1,2,1,0,0,...,1,0,1,0,1,1,0,0,1,0
6,24,2835,3,4,53,1,1,1,0,0,...,0,0,1,1,0,0,1,0,0,0
7,36,6948,2,2,35,1,1,1,1,0,...,0,0,1,0,0,0,0,1,1,0
8,12,3059,2,4,61,1,1,1,0,0,...,0,0,1,1,0,1,0,0,0,0
9,30,5234,4,2,28,2,1,2,1,0,...,0,0,1,1,0,0,0,1,0,0


## 2.2.2